In [142]:
import pandas as pd 

path = r'C:\Users\user\Desktop\HELP\clean1209-1.xlsx'
data = pd.read_excel(path)
data.head()


,計畫主持人,號,機關名稱,計畫名稱,核定金額,執行期限(起),執行期限(迄),Email,職稱,學門名稱2,研究年期,年份
0,丁一顧,SS2,臺北市立大學教育行政與評鑑研究所,國小校長優勢本位領導教練：理論、工具與實踐之研究,1591000,109/08/01,111/07/31,tim@utaipei.edu.tw,教授,教育行政,2,109
1,丁一顧,SS2,臺北市立大學教育行政與評鑑研究所,校長正向教學視導實施對教師教學效能影響之研究,1216000,107/08/01,109/07/31,tim@utaipei.edu.tw,教授,教育行政,2,107
2,丁志音,SS3,國立臺灣大學公共衛生學院健康政策與管理研究所,衛生行為中的「界線」與「慣行」：從預防感染到疫情控制,2975000,20120801,20150731,chihyin@ntu.edu.tw,NaN,社會問題與控制,3,101
3,丁國桓,1,國立高雄科技大學管理學院海洋事務與產業管理研究所,專題導向學習運用於大學生實踐社會責任概念建構之行動研究,582000,108/08/01,109/07/31,dgh0809@nkust.edu.tw,助理教授,課程與教學,1,108
4,丁崇德,1,長榮大學觀光與餐飲管理學系,衡量居民健康風險知覺改善的社區安全品質效益與績效：社區治安觀點,460000,110/08/01,111/07/31,ctting@mail.cjcu.edu.tw,副教授,都市及區域,1,110


In [ ]:
data['號'] = data['號'].str.replace('SSS','1').replace('sss','1')
data['研究年期'] = data['號'].str.extract('(\d+)')


In [151]:
data['學校'] = data['機關名稱'].str.split('大學',expand=True)[0]
data['系所'] = data['機關名稱'].str.split('大學',expand=True)[1]


data['學校'] = data['學校']+'大學'


In [157]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
#pd.set_option('display.max_columns', df.shape[0]+1)
#pd.set_option('display.max_rows', None)


def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """ param df_1: 計畫提供
        param df_2: SRDA全
        param key1: 變項名稱設定為 「原始題目」
        param key2: 變項名稱設定為 「原始題目」
        param threshold: 比對精準度 設定為90
        param limit: the amount of matches that will get returned, these are sorted high to low:return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))
    df_1['模糊比對'] = m
    
    m2 = df_1['模糊比對'].apply(lambda x: [i[0] for i in x if i[1] >= threshold][0] if len([i[0] for i in x if i[1] >= threshold]) > 0 else '')
    df_1['模糊比對'] = m2
    return df_1
    


SRDA_path = r'C:\Users\user\Desktop\HELP\學校地區\學校地區總表.xlsx'
survey_path = r'C:\Users\user\Desktop\HELP\clean1209-2.xlsx'


SRDA = pd.read_excel(SRDA_path).astype(str)
survey= pd.read_excel(survey_path).astype(str)


df = fuzzy_merge(survey, SRDA, '學校', '學校名稱', threshold=90)

##################################################################一般情況 85.88
##################################################################純英文95.66
df

,計畫主持人,號,學校名稱,計畫名稱,核定金額,執行期限(起),執行期限(迄),Email,職稱,學門名稱2,研究年期,年份,學校,系所,模糊比對
0,丁一顧,SS2,臺北市立大學教育行政與評鑑研究所,國小校長優勢本位領導教練：理論、工具與實踐之研究,1591000,109/08/01,111/07/31,tim@utaipei.edu.tw,教授,教育行政,2,109,臺北市立大學,教育行政與評鑑研究所,臺北市立大學
1,丁一顧,SS2,臺北市立大學教育行政與評鑑研究所,校長正向教學視導實施對教師教學效能影響之研究,1216000,107/08/01,109/07/31,tim@utaipei.edu.tw,教授,教育行政,2,107,臺北市立大學,教育行政與評鑑研究所,臺北市立大學
2,丁志音,SS3,國立臺灣大學公共衛生學院健康政策與管理研究所,衛生行為中的「界線」與「慣行」：從預防感染到疫情控制,2975000,20120801,20150731,chihyin@ntu.edu.tw,nan,社會問題與控制,3,101,國立臺灣大學,公共衛生學院健康政策與管理研究所,國立臺灣大學
3,丁國桓,1,國立高雄科技大學管理學院海洋事務與產業管理研究所,專題導向學習運用於大學生實踐社會責任概念建構之行動研究,582000,108/08/01,109/07/31,dgh0809@nkust.edu.tw,助理教授,課程與教學,1,108,國立高雄科技大學,管理學院海洋事務與產業管理研究所,國立高雄科技大學
4,丁崇德,1,長榮大學觀光與餐飲管理學系,衡量居民健康風險知覺改善的社區安全品質效益與績效：社區治安觀點,460000,110/08/01,111/07/31,ctting@mail.cjcu.edu.tw,副教授,都市及區域,1,110,長榮大學,觀光與餐飲管理學系,長榮大學
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569,蘇靖淑,1,銘傳大學餐旅管理學系,東協國家旅客知多少？計劃行為理論結合電子口碑、目的地意象探討東協國家來台旅客行為之研究,743000,106/08/01,107/07/31,sallysu@mail.mcu.edu.tw,副教授,休閒遊憩,1,106,銘傳大學,餐旅管理學系,銘傳大學
1570,蘇靖淑,nan,景文科技大學餐飲管理系,加盟連鎖總部評量工具發展之研究－以餐飲連鎖業為例,530000,20110801,20120731,sallysu@just.edu.tw,nan,休閒遊憩,1,100,景文科技大學,餐飲管理系,景文科技大學
1571,蘇蘅,SS2,國立政治大學新聞學系,社群媒體、來源偏好、與網路新聞分享：跨世代新聞選擇與社會可信度的相關性,1315000,104/08/01,106/07/31,herngsu@nccu.edu.tw,教授,新聞,2,104,國立政治大學,新聞學系,國立政治大學
1572,蘇蘅,SS2,國立政治大學新聞學系,數位時代的媒介倫理：重探新聞人員網路使用、新聞產製與專業評價的關係,1272000,20130801,20150731,herngsu@nccu.edu.tw,nan,新聞,2,102,國立政治大學,新聞學系,國立政治大學


In [159]:
import pandas as pd
A_path = r'C:\Users\user\Desktop\HELP\clean1209-3.xlsx'  #GRB爬蟲結果路徑
B_path = r'C:\Users\user\Desktop\HELP\學校地區\學校地區總表.xlsx'  #MOST爬蟲結果路徑

A = pd.DataFrame(pd.read_excel(A_path))  #GRB名冊
send = pd.DataFrame(pd.read_excel(B_path))  #MOST爬蟲結果



merge1 = pd.merge(A,
                  send,
                  left_on='模糊比對',
                  right_on='學校名稱',
                  how='outer')

merge1

,計畫主持人,號,學校名稱_x,計畫名稱,核定金額,執行期限(起),執行期限(迄),Email,職稱,學門名稱2,研究年期,年份,學校,系所,模糊比對,公私立,體制,學校名稱_y,學校地址,學校網址,地區
0,丁一顧,SS2,臺北市立大學教育行政與評鑑研究所,國小校長優勢本位領導教練：理論、工具與實踐之研究,1591000.0,109/08/01,111/07/31,tim@utaipei.edu.tw,教授,教育行政,2.0,109.0,臺北市立大學,教育行政與評鑑研究所,臺北市立大學,公立,一般大學,臺北市立大學,臺北市中正區愛國西路1號,http://www.utaipei.edu.tw/,北部
1,丁一顧,SS2,臺北市立大學教育行政與評鑑研究所,校長正向教學視導實施對教師教學效能影響之研究,1216000.0,107/08/01,109/07/31,tim@utaipei.edu.tw,教授,教育行政,2.0,107.0,臺北市立大學,教育行政與評鑑研究所,臺北市立大學,公立,一般大學,臺北市立大學,臺北市中正區愛國西路1號,http://www.utaipei.edu.tw/,北部
2,王珮玲,1,臺北市立大學幼兒教育學系,高低努力控制幼兒在挫折情境下之情緒調節策略,677000.0,109/08/01,110/07/31,peilinwang2001@yahoo.com.tw,教授,幼兒教育,1.0,109.0,臺北市立大學,幼兒教育學系,臺北市立大學,公立,一般大學,臺北市立大學,臺北市中正區愛國西路1號,http://www.utaipei.edu.tw/,北部
3,何希慧,SS2,臺北市立大學教育行政與評鑑研究所,擺盪在教學與研究間: 談我國高等教育推動教學與研究連結的可行性與挑戰,1695000.0,110/08/01,112/07/31,shihuei@utaipei.edu.tw,教授,教育行政,2.0,110.0,臺北市立大學,教育行政與評鑑研究所,臺北市立大學,公立,一般大學,臺北市立大學,臺北市中正區愛國西路1號,http://www.utaipei.edu.tw/,北部
4,何希慧,1,臺北市立大學教育行政與評鑑研究所,從菁英人才永續發展看我國博士後訓練之問題與挑戰,666000.0,109/08/01,110/07/31,shihuei@utaipei.edu.tw,教授,教育行政,1.0,109.0,臺北市立大學,教育行政與評鑑研究所,臺北市立大學,公立,一般大學,臺北市立大學,臺北市中正區愛國西路1號,http://www.utaipei.edu.tw/,北部
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,公立,技專校院,國立澎湖科技大學,澎湖縣馬公市六合路300號,http://www.npu.edu.tw,外島
1623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,公立,技專校院,國立臺東專科學校,臺東縣臺東市正氣北路911號,http://www.ntc.edu.tw/,東部
1624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,私立,技專校院,大漢學校財團法人大漢技術學院,花蓮縣新城鄉大漢村樹人街1號,http://www.dahan.edu.tw/,東部
1625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,私立,技專校院,蘭陽技術學院,宜蘭縣頭城鎮復興路79號,http://www.fit.edu.tw/,東部


In [170]:
extra = merge1.groupby('計畫主持人').sum()

extra


merge2 = pd.merge(merge1,
                  extra,
                  left_on='計畫主持人',
                  right_on='計畫主持人',
                  how='outer')

merge2


,計畫主持人,號,學校名稱_x,計畫名稱,核定金額_x,執行期限(起),執行期限(迄),Email,職稱,學門名稱2,研究年期_x,年份_x,學校,系所,模糊比對,公私立,體制,學校名稱_y,學校地址,學校網址,地區,核定金額_y,研究年期_y,年份_y
0,丁一顧,SS2,臺北市立大學教育行政與評鑑研究所,國小校長優勢本位領導教練：理論、工具與實踐之研究,1591000.0,109/08/01,111/07/31,tim@utaipei.edu.tw,教授,教育行政,2.0,109.0,臺北市立大學,教育行政與評鑑研究所,臺北市立大學,公立,一般大學,臺北市立大學,臺北市中正區愛國西路1號,http://www.utaipei.edu.tw/,北部,2807000.0,4.0,216.0
1,丁一顧,SS2,臺北市立大學教育行政與評鑑研究所,校長正向教學視導實施對教師教學效能影響之研究,1216000.0,107/08/01,109/07/31,tim@utaipei.edu.tw,教授,教育行政,2.0,107.0,臺北市立大學,教育行政與評鑑研究所,臺北市立大學,公立,一般大學,臺北市立大學,臺北市中正區愛國西路1號,http://www.utaipei.edu.tw/,北部,2807000.0,4.0,216.0
2,王珮玲,1,臺北市立大學幼兒教育學系,高低努力控制幼兒在挫折情境下之情緒調節策略,677000.0,109/08/01,110/07/31,peilinwang2001@yahoo.com.tw,教授,幼兒教育,1.0,109.0,臺北市立大學,幼兒教育學系,臺北市立大學,公立,一般大學,臺北市立大學,臺北市中正區愛國西路1號,http://www.utaipei.edu.tw/,北部,3396000.0,5.0,311.0
3,王珮玲,SS2,國立暨南國際大學社會政策與社會工作學系（所）,親密伴侶經濟虐待之現象與回應,1408000.0,20130801,20150731,plwang@ncnu.edu.tw,NaN,社會問題與控制,2.0,102.0,國立暨南國際大學,社會政策與社會工作學系（所）,國立暨南國際大學,公立,一般大學,國立暨南國際大學,南投縣埔里鎮大學路1號,http://www.ncnu.edu.tw,中部,3396000.0,5.0,311.0
4,王珮玲,SS2,國立暨南國際大學社會政策與社會工作學系（所）,親密伴侶跟蹤行為之研究：現象、影響與回應,1311000.0,20110801,20130731,plwang@ncnu.edu.tw,NaN,社會問題與控制,2.0,100.0,國立暨南國際大學,社會政策與社會工作學系（所）,國立暨南國際大學,公立,一般大學,國立暨南國際大學,南投縣埔里鎮大學路1號,http://www.ncnu.edu.tw,中部,3396000.0,5.0,311.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,公立,技專校院,國立澎湖科技大學,澎湖縣馬公市六合路300號,http://www.npu.edu.tw,外島,NaN,NaN,NaN
1623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,公立,技專校院,國立臺東專科學校,臺東縣臺東市正氣北路911號,http://www.ntc.edu.tw/,東部,NaN,NaN,NaN
1624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,私立,技專校院,大漢學校財團法人大漢技術學院,花蓮縣新城鄉大漢村樹人街1號,http://www.dahan.edu.tw/,東部,NaN,NaN,NaN
1625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,私立,技專校院,蘭陽技術學院,宜蘭縣頭城鎮復興路79號,http://www.fit.edu.tw/,東部,NaN,NaN,NaN


In [171]:
SaveFile_Path = r'C:\Users\user\Desktop\HELP'  #要儲存檔案的路徑
SaveFile_Name = r'clean1209-5-累加'  #儲存的檔名

merge2.to_excel(SaveFile_Path + '\\' + SaveFile_Name +
                   ".xlsx",
                   encoding="utf_8_sig",
                   index=False)
print('比對成功')

比對成功
